<a href="https://colab.research.google.com/github/susansunshin/Project-2/blob/main/Proejct_2_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 2 Part 2

Susan Shin

##Load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import set_config
set_config(display='diagram')

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
from google.colab import drive
drive.mount('/content/drive')

filename = "/content/sample_data/wines_SPA 3.csv"
df = pd.read_csv(filename)
df.head()

Mounted at /content/drive


,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


##Explore Data

In [5]:
#Make Copy
df_ml = df

In [6]:
#Remove num_reviews
df_ml = df_ml.drop(columns=["num_reviews"])
df_ml.head()

,winery,wine,year,rating,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [7]:
#Check for Dupes
df_ml.duplicated().sum()

5452

In [8]:
#Drop Dupes
df_ml = df_ml.drop_duplicates()
df_ml.duplicated().sum()

0

In [9]:
#Check for NAs
df_ml.isna().sum()

winery       0
wine         0
year         2
rating       0
country      0
region       0
price        0
type       106
body       271
acidity    271
dtype: int64

In [10]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2048 entries, 0 to 6100
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   winery   2048 non-null   object 
 1   wine     2048 non-null   object 
 2   year     2046 non-null   object 
 3   rating   2048 non-null   float64
 4   country  2048 non-null   object 
 5   region   2048 non-null   object 
 6   price    2048 non-null   float64
 7   type     1942 non-null   object 
 8   body     1777 non-null   float64
 9   acidity  1777 non-null   float64
dtypes: float64(4), object(6)
memory usage: 176.0+ KB


In [11]:
#Check for consistencies for object values
pd.set_option('display.max_column', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', -1)
pd.set_option('display.max_rows', None)

dtypes = df_ml.dtypes
str_cols = dtypes[dtypes=='object'].index
str_cols
  
for col in str_cols:
  print(f"- Column= {col}")
  print(df[col].value_counts(dropna=False))
  print('\n\n')

- Column= winery
Contino                                             457
Artadi                                              261
La Rioja Alta                                       254
Sierra Cantabria                                    237
Matarromera                                         232
Vina Pedrosa                                        230
Imperial                                            228
Losada                                              227
Portal del Priorat                                  225
Ramon Bilbao                                        225
Sei Solo                                            225
Matsu                                               224
Bodegas La Horra                                    223
Campillo                                            223
Lustau                                              223
La Vicalanda                                        223
Conreria d'Scala Dei                                223
Mar de Frades                  

The object in year is N.V. - which means Non Vintage. So this is a valid value.

Country is all Spain, no need for this column, will delete.

In [12]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2048 entries, 0 to 6100
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   winery   2048 non-null   object 
 1   wine     2048 non-null   object 
 2   year     2046 non-null   object 
 3   rating   2048 non-null   float64
 4   country  2048 non-null   object 
 5   region   2048 non-null   object 
 6   price    2048 non-null   float64
 7   type     1942 non-null   object 
 8   body     1777 non-null   float64
 9   acidity  1777 non-null   float64
dtypes: float64(4), object(6)
memory usage: 176.0+ KB


In [13]:
del df_ml['year']
df_ml.head()

,winery,wine,rating,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,4.9,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,4.9,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,4.8,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,4.8,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,4.8,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [14]:
#Check Numerical values for outliers
df_ml["rating"].value_counts()

4.3    706
4.4    484
4.5    281
4.2    228
4.6    191
4.7    112
4.8     44
4.9      2
Name: rating, dtype: int64

In [15]:
df_ml["body"].value_counts()

4.0    1003
5.0     634
3.0     106
2.0      34
Name: body, dtype: int64

In [16]:
df_ml["acidity"].value_counts()

3.0    1672
2.0      70
1.0      35
Name: acidity, dtype: int64

Data is now ready for preprocessing!

##Preprocessing

In [17]:
#Split Data
X = df_ml.drop(columns=["rating"])
y = df_ml["rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [18]:
X_train.isna().sum()

winery       0
wine         0
country      0
region       0
price        0
type        79
body       207
acidity    207
dtype: int64

In [19]:
#Create selectors
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

In [20]:
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

impute_mean = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')

In [21]:
#Create pipes
num_pipe = make_pipeline(impute_mean, scaler)
cat_pipe = make_pipeline(freq_imputer, ohe)

In [22]:
#Tupes
num_tup = (num_pipe, num_selector)
cat_tup = (cat_pipe, cat_selector)

In [23]:
#Create preprocessor
preprocessor = make_column_transformer(cat_tup, num_tup, remainder = 'drop')
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8a8310>)])

##Random Forest Model

In [24]:
rf = RandomForestRegressor(random_state = 42)
rf_pipe = make_pipeline(preprocessor, rf)
rf_pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8a8310>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [25]:
print(rf_pipe.score(X_train, y_train))
print(rf_pipe.score(X_test, y_test))

0.933583076323508
0.4878609575174797


###Tune

In [26]:
#Get Parameters
rf_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer', ColumnTransformer(transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sparse=False))]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                   ('pipeline-2',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer()),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                 

In [27]:
#Find max depth
est_depths = [estimator.get_depth() for estimator in rf.estimators_]
max(est_depths)

97

In [28]:
#Run grid search
rf_param_grid = {'randomforestregressor__max_depth': range(1,97)}

rf_pipe_gs = GridSearchCV(rf_pipe, rf_param_grid)
rf_pipe_gs.fit(X_train, y_train)
print('Best Parameters:')
print(rf_pipe_gs.best_params_)
best_pipe = rf_pipe_gs.best_estimator_
print(f'Accuracy of best model is: {best_pipe.score(X_test, y_test)}')

Best Parameters:
{'randomforestregressor__max_depth': 26}
Accuracy of best model is: 0.5043140224966872


In [29]:
#Create tuned pipe
rf_tuned = RandomForestRegressor(max_depth = 26, random_state = 42)
rf_pipe_tuned = make_pipeline(preprocessor, rf)
rf_pipe_tuned.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8a8310>)])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

###Score

In [52]:
def eval_regression(true, pred):

  mae = round(mean_absolute_error(true, pred),5)
  mse = round(mean_squared_error(true, pred),5)
  rmse = round(np.sqrt(mse),5)
  r2 = round(r2_score(true, pred),5)

  print(f'MAE {mae},\n MSE {mse},\n RMSE: {rmse},\n R^2: {r2}\n ')

In [53]:
rf_test_pred_tuned = rf_pipe_tuned.predict(X_test)
rf_train_pred_tuned = rf_pipe_tuned.predict(X_train)

In [54]:
eval_regression(y_train, rf_train_pred_tuned)
eval_regression(y_test, rf_test_pred_tuned)

MAE 0.02931,
 MSE 0.00146,
 RMSE: 0.03821,
 R^2: 0.93358
 
MAE 0.07853,
 MSE 0.01042,
 RMSE: 0.10208,
 R^2: 0.48786
 


##KNeighbors Model

In [48]:
#Create Pipe
kn = KNeighborsRegressor()
kn_pipe = make_pipeline(preprocessor, kn)
kn_pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8a8310>)])),
                ('kneighborsregressor', KNeighborsRegressor())])

In [42]:
#Check score
print(kn_pipe.score(X_train, y_train))
print(kn_pipe.score(X_test, y_test))

0.676487036522272
0.4356962153467152


###Tune

In [43]:
#Get Parameters
kn_pipe.get_params()

{'memory': None,
 'steps': [('columntransformer', ColumnTransformer(transformers=[('pipeline-1',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sparse=False))]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                   ('pipeline-2',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer()),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                 

In [49]:
#Run grid search
kn_param_grid = {"kneighborsregressor__n_neighbors": range(1,20)}

kn_pipe_gs = GridSearchCV(kn_pipe, kn_param_grid)
kn_pipe_gs.fit(X_train, y_train)
print('Best Parameters:')
print(kn_pipe_gs.best_params_)
best_pipe = kn_pipe_gs.best_estimator_
print(f'Accuracy of best model is: {best_pipe.score(X_test, y_test)}')

Best Parameters:
{'kneighborsregressor__n_neighbors': 4}
Accuracy of best model is: 0.4205826495010223


In [45]:
#Create tuned model
kn_tuned = KNeighborsRegressor(n_neighbors = 4)
kn_pipe_tuned = make_pipeline(preprocessor, kn_tuned)
kn_pipe_tuned.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8948d0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f73ac8a8310>)])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=4))])

In [46]:
print(kn_pipe_tuned.score(X_train, y_train))
print(kn_pipe_tuned.score(X_test, y_test))

0.7031340115054666
0.4205826495010223


###Score

In [50]:
kn_test_pred_tuned = kn_pipe_tuned.predict(X_test)
kn_train_pred_tuned = kn_pipe_tuned.predict(X_train)

In [58]:
eval_regression(y_train, kn_train_pred_tuned)
eval_regression(y_test, kn_test_pred_tuned)

MAE 0.06195,
 MSE 0.00654,
 RMSE: 0.08087,
 R^2: 0.70313
 
MAE 0.08408,
 MSE 0.01179,
 RMSE: 0.10858,
 R^2: 0.42058
 


#Conclusion

In [59]:
#View both scores togehter
print("Random Forest")
eval_regression(y_train, rf_train_pred_tuned)
eval_regression(y_test, rf_test_pred_tuned)

Random Forest
MAE 0.02931,
 MSE 0.00146,
 RMSE: 0.03821,
 R^2: 0.93358
 
MAE 0.07853,
 MSE 0.01042,
 RMSE: 0.10208,
 R^2: 0.48786
 


In [60]:
print("KNNeighbors")
eval_regression(y_train, kn_train_pred_tuned)
eval_regression(y_test, kn_test_pred_tuned)

KNNeighbors
MAE 0.06195,
 MSE 0.00654,
 RMSE: 0.08087,
 R^2: 0.70313
 
MAE 0.08408,
 MSE 0.01179,
 RMSE: 0.10858,
 R^2: 0.42058
 


Random Forest would be my recomensation for this exercise as it has a higher R2 score and a lower MAE and RMSE score.